In [47]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

seed_number = 123

In [24]:
main_dir = os.path.abspath(os.getcwd())

main_dir

'/home/pedro/Downloads/xgboost'

### ETAPA 1: lendo a base de dados

In [25]:
train = pd.read_csv(f'{main_dir}/input/train.csv')
test = pd.read_csv(f'{main_dir}/input/test.csv')
train.shape, test.shape

((113754, 24), (28439, 24))

In [26]:
data = pd.concat([train,test],sort=True)
data.columns

Index(['Cloud3pm', 'Cloud9am', 'Date', 'Evaporation', 'Humidity3pm',
       'Humidity9am', 'Id', 'Location', 'MaxTemp', 'MinTemp', 'Pressure3pm',
       'Pressure9am', 'RainToday', 'RainTomorrow', 'Rainfall', 'Sunshine',
       'Temp3pm', 'Temp9am', 'WindDir3pm', 'WindDir9am', 'WindGustDir',
       'WindGustSpeed', 'WindSpeed3pm', 'WindSpeed9am'],
      dtype='object')

In [27]:
data.shape, data.columns

((142193, 24),
 Index(['Cloud3pm', 'Cloud9am', 'Date', 'Evaporation', 'Humidity3pm',
        'Humidity9am', 'Id', 'Location', 'MaxTemp', 'MinTemp', 'Pressure3pm',
        'Pressure9am', 'RainToday', 'RainTomorrow', 'Rainfall', 'Sunshine',
        'Temp3pm', 'Temp9am', 'WindDir3pm', 'WindDir9am', 'WindGustDir',
        'WindGustSpeed', 'WindSpeed3pm', 'WindSpeed9am'],
       dtype='object'))

In [42]:
data.head()

,Cloud3pm,Cloud9am,Evaporation,Humidity3pm,Humidity9am,MaxTemp,MinTemp,Pressure3pm,Pressure9am,RainToday,RainTomorrow,Rainfall,Temp3pm,Temp9am,WindGustSpeed,WindSpeed3pm,WindSpeed9am
0,NaN,NaN,NaN,25.0,44.0,25.1,7.4,1007.8,1010.6,No,No,0.0,24.3,17.2,44.0,22.0,4.0
1,2.0,NaN,NaN,30.0,38.0,25.7,12.9,1008.7,1007.6,No,No,0.0,23.2,21.0,46.0,26.0,19.0
2,NaN,NaN,NaN,16.0,45.0,28.0,9.2,1012.8,1017.6,No,No,0.0,26.5,18.1,24.0,9.0,11.0
3,NaN,NaN,NaN,23.0,55.0,29.7,14.6,1005.4,1009.2,No,No,0.2,28.9,20.6,56.0,24.0,19.0
4,NaN,NaN,NaN,19.0,48.0,26.7,7.7,1010.1,1013.4,No,No,0.0,25.5,16.3,35.0,17.0,6.0


In [35]:
data.describe()

,Cloud3pm,Cloud9am,Evaporation,Humidity3pm,Humidity9am,MaxTemp,MinTemp,Pressure3pm,Pressure9am,Rainfall,Sunshine,Temp3pm,Temp9am,WindGustSpeed,WindSpeed3pm,WindSpeed9am
count,84693.000000,88162.000000,81093.000000,137286.000000,139270.000000,140480.00000,140319.000000,127018.000000,127044.000000,140787.000000,73982.000000,138163.000000,140131.000000,131682.000000,138256.000000,139732.000000
mean,4.499250,4.431161,5.472516,51.449288,68.826833,23.23512,12.184824,1015.257963,1017.654577,2.349974,7.630540,21.693183,16.987066,39.970520,18.631141,13.990496
std,2.719752,2.886594,4.189132,20.807310,19.063650,7.11450,6.403879,7.035411,7.104867,8.465173,3.781729,6.937784,6.496012,13.578201,8.798096,8.886210
min,0.000000,0.000000,0.000000,0.000000,0.000000,-4.80000,-8.500000,977.100000,980.500000,0.000000,0.000000,-5.400000,-7.200000,6.000000,0.000000,0.000000
25%,2.000000,1.000000,2.600000,37.000000,57.000000,17.90000,7.600000,1010.400000,1013.000000,0.000000,4.900000,16.600000,12.300000,31.000000,13.000000,7.000000
50%,5.000000,5.000000,4.800000,52.000000,70.000000,22.60000,12.000000,1015.200000,1017.600000,0.000000,8.500000,21.100000,16.700000,39.000000,19.000000,13.000000
75%,7.000000,7.000000,7.400000,66.000000,83.000000,28.30000,16.800000,1020.000000,1022.400000,0.800000,10.700000,26.400000,21.600000,48.000000,24.000000,19.000000
max,9.000000,9.000000,145.000000,100.000000,100.000000,48.10000,33.900000,1039.600000,1041.000000,371.000000,14.500000,46.700000,40.200000,135.000000,87.000000,130.000000


In [22]:
data.describe(include=['O'])

,Data,Localizacao,ChoveHoje,ChoveAmanha,DirecaoVento15h,DirecaoVento9h,RajadadeVentoDirecao
count,142193,142193,140787,142193,138415,132180,132863
unique,3436,49,2,2,16,16,16
top,2017-05-10,Canberra,No,No,SE,N,W
freq,49,3418,109332,110316,10663,11393,9780


In [28]:
# checando quais atributos são numéricos (não consegui exibir o dataframe inteiro no vscode)
numerics =  data.select_dtypes(include = np.number).columns.to_list()
print(numerics)

"""
17 atributos numéricos no total
"""

['Cloud3pm', 'Cloud9am', 'Evaporation', 'Humidity3pm', 'Humidity9am', 'Id', 'MaxTemp', 'MinTemp', 'Pressure3pm', 'Pressure9am', 'Rainfall', 'Sunshine', 'Temp3pm', 'Temp9am', 'WindGustSpeed', 'WindSpeed3pm', 'WindSpeed9am']


In [30]:
#checando atributos categoricos
categorical = data.select_dtypes('object').columns.to_list()
print(categorical)

"""
7 atributos categóricos
"""

['Date', 'Location', 'RainToday', 'RainTomorrow', 'WindDir3pm', 'WindDir9am', 'WindGustDir']


### Etapa 2: Limpeza e preparação dos Dados

In [ ]:
# o ID é inútil pq só atribui um valor unico pra a instancia do dado então vou dropar
data.drop("Id", axis=1, inplace=True)
data

In [34]:
# verificando valores nulos
data.isna().sum().sort_values(ascending = False)

Sunshine         66805
Evaporation      59694
Cloud3pm         56094
Cloud9am         52625
Pressure3pm      13769
Pressure9am      13743
WindDir9am        9660
WindGustDir       9163
WindGustSpeed     9105
WindDir3pm        3670
Humidity3pm       3501
Temp3pm           2624
WindSpeed3pm      2531
Humidity9am       1517
WindSpeed9am      1055
Temp9am            656
MinTemp            468
MaxTemp            307
Rainfall             0
RainTomorrow         0
Location             0
Date                 0
RainToday            0
dtype: int64

In [33]:
# tem 1406 nulos em RainToday. Pode afetar no treinamento. Dropando...
data.dropna(subset = ["RainToday"], inplace = True)

In [36]:
# Dropando Date e Location por serem categoricas e não contribuírem muito pra o resultado (não sem muito pré-processamento e análise geográfica e climatológica que eu não tenho conhecimento)
# Dropando Sunshine por estar mais da metade nula, então não enxergo a imputação como solução boa (Pode produzir dados irreais demais)
# Dropando WindDir(todos) por envolver análise geográfica que eu não tenho conhecimento (fora que são categóricos)
data.drop(['Date', 'Sunshine', 'Location', 'WindDir3pm', 'WindDir9am', 'WindGustDir'], axis=1, inplace=True) 

In [45]:
# checando de novo pra ver como tá a base de dados
data.isnull().sum()

Cloud3pm         0
Cloud9am         0
Evaporation      0
Humidity3pm      0
Humidity9am      0
MaxTemp          0
MinTemp          0
Pressure3pm      0
Pressure9am      0
RainToday        0
RainTomorrow     0
Rainfall         0
Temp3pm          0
Temp9am          0
WindGustSpeed    0
WindSpeed3pm     0
WindSpeed9am     0
dtype: int64

In [44]:
# Agora será necessário fazer imputação dos resultados nulos

# Evaporation eu considero um dado muito importante pra dropar, apesar da quantidade de nulos. Acho que se imputar a moda fica mais próximo da realidade
# Cloud e Humidity por serem inteiras
data['Evaporation'].fillna(data['Evaporation'].mode()[0], inplace=True)
data['Cloud3pm'].fillna(data['Cloud3pm'].mode()[0], inplace=True)
data['Cloud9am'].fillna(data['Cloud9am'].mode()[0], inplace=True)
data['Humidity3pm'].fillna(data['Humidity3pm'].mode()[0], inplace=True)
data['Humidity9am'].fillna(data['Humidity9am'].mode()[0], inplace=True)

# Temperaturas e Pressão podem ser imputadas pela média por serem continuas
# Velocidades também acho adequado usar média por serem bem menos dados, então se o preenchimento(caso incorreto) não impacta tanto o resultado
data['MaxTemp'].fillna(data['MaxTemp'].mean(), inplace=True)
data['MinTemp'].fillna(data['MinTemp'].mean(), inplace=True)
data['Temp3pm'].fillna(data['Temp3pm'].mean(), inplace=True)
data['Temp9am'].fillna(data['Temp9am'].mean(), inplace=True)
data['Pressure3pm'].fillna(data['Pressure3pm'].mean(), inplace=True)
data['Pressure9am'].fillna(data['Pressure9am'].mean(), inplace=True)
data['WindGustSpeed'].fillna(data['WindGustSpeed'].mean(), inplace=True)
data['WindSpeed3pm'].fillna(data['WindSpeed3pm'].mean(), inplace=True)
data['WindSpeed9am'].fillna(data['WindSpeed9am'].mean(), inplace=True)

# pronto agora todos os valores nulos estão preenchidos 
data.isnull().sum()

#não houve necessidade de preencher a variável alvo pois ela já veio completamente preenchida


In [49]:
print(data['RainTomorrow'].value_counts())
print(data['RainTomorrow'].value_counts()/data['RainTomorrow'].count())

No     109586
Yes     31201
Name: RainTomorrow, dtype: int64
No     0.778382
Yes    0.221618
Name: RainTomorrow, dtype: float64


### Etapa 3: Divisão treino-teste e normalização dos dados

AQUI EU VOU USAR O MÉTODO HOLDOUT (70% treino/ 30% teste)

In [51]:
# dividindo o dataset entre amostras e labels
alldata_x = data.drop('RainTomorrow', axis=1)
alldata_y = data['RainTomorrow']
alldata_x.shape, alldata_y.shape

((140787, 16), (140787,))

In [ ]:
# Agora dividir entre treino e teste
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

np.random.seed(seed_number)
# Divisão Holdout 70% treino / 30% teste com estratificação
X_train, X_test, y_train, y_test = train_test_split(alldata_x, alldata_y, test_size = 0.3, stratify=y, random_state = seed_number)
X_train.shape, X_test.shape, y_train.shape, y_test.shape